In [ ]:
#| hide
from raincell import *

# raincell

> A user-friendly Python library for transforming commercial microwave links into rain estimation insights. Built with nbdev and Jupyter notebooks, this tool offers robust scientific and visualization capabilities, tailored for researchers in the field.

This file will become your README and also the index of your documentation.

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install -U git+ssh://git@github.com/rainsmore/raincell.git
```

or from [conda][conda] (not available yet)

```sh
$ conda install -c rainsmore raincell
```

or from [pypi][pypi] (not available yet)


```sh
$ pip install raincell
```


[repo]: https://github.com/rainsmore/raincell
[docs]: https://rainsmore.github.io/raincell/
[pypi]: https://pypi.org/project/raincell/
[conda]: https://anaconda.org/rainsmore/raincell

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install raincell in Development mode

make sure raincell package is installed in development mode
```sh
$ pip install -e .
```

You may also need to install the pre-commit hooks. If you are using Jupyter Notebooks or Jupyter Lab as your IDE, you only need to run:

```sh
nbdev_install_hooks
```

However, if you are using another IDE such as VS Code, you will need to run:

```sh
pre-commit install
```

Check the official nbdev documentation for more information on [nbdev_install_hooks](https://nbdev.fast.ai/tutorials/git_friendly_jupyter.html) or on [Pre-Commit Hooks](https://nbdev.fast.ai/tutorials/pre_commit.html).

Then make changes under nbs/ directory
```bash
...
```
compile to have changes apply to raincell
```sh 
$ nbdev_prepare
```

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/rainsmore/raincell
[docs]: https://rainsmore.github.io/raincell/
[pypi]: https://pypi.org/project/raincell/
[conda]: https://anaconda.org/rainsmore/raincell

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2